# Travel Demo

Diese Demoanwendung stellt mehrere Dienste bereit, die in drei Namespaces gruppiert sind.

![](https://kiali.io/images/tutorial/02-02-travels-demo-design.png)

Quelle: https://kiali.io/docs/tutorials/travels/02-install-travel-demo/
- - - 

In [ ]:
%%bash
kubectl create namespace travel-agency
kubectl create namespace travel-portal
kubectl create namespace travel-control
kubectl label  namespace travel-agency istio-injection=enabled
kubectl label  namespace travel-portal istio-injection=enabled
kubectl label  namespace travel-control istio-injection=enabled
kubectl apply -f https://raw.githubusercontent.com/kiali/demos/master/travels/travel_agency.yaml -n travel-agency
kubectl apply -f https://raw.githubusercontent.com/kiali/demos/master/travels/travel_portal.yaml -n travel-portal
kubectl apply -f https://raw.githubusercontent.com/kiali/demos/master/travels/travel_control.yaml -n travel-control

##  Demo-Anwendung

### Reiseportal-Namespace

Die Travel Demo-Anwendung simuliert zwei Geschäftsdomänen, die in unterschiedlichen Namespaces organisiert sind.

In einem ersten Namensraum namens Reiseportal werden mehrere Reiseshops bereitgestellt, in denen Benutzer Flüge, Hotels, Autos oder Versicherungen suchen und buchen können.

Die Shop-Applikationen können sich je nach Anforderungsmerkmalen wie Kanal (Web oder Mobile) oder Benutzer (neu oder bestehend) unterschiedlich verhalten.

Diese Workloads können unterschiedliche Arten von Datenverkehr erzeugen, um unterschiedliche reale Szenarien zu imitieren.

Alle Portale verbrauchen einen Dienst namens travels , der im Reisebüro- Namespace bereitgestellt wird.

### Reisebüro-Namespace

Ein zweiter Namespace mit dem Namen Reisebüro hostet eine Reihe von Diensten, die erstellt wurden, um Angebote für Reisen bereitzustellen.

Ein Hauptreisedienst wird der geschäftliche Einstiegspunkt für das Reisebüro sein. Es erhält einen Zielort und einen Benutzer als Parameter und berechnet alle Elemente, die ein Reisebudget ausmachen: Flugpreis, Unterkunft, Autoreservierung und Reiseversicherung.

Jeder Dienst kann ein unabhängiges Angebot erstellen, und der Reisedienst muss diese dann zu einer einzigen Antwort zusammenfassen.

Darüber hinaus können einige Benutzer, wie registrierte Benutzer, Zugang zu Sonderrabatten haben, die ebenfalls von einem externen Dienst verwaltet werden.

- - - 
## Öffnen Travel Demo für den Zugriff von Aussen

### DNS Eintrag oder /etc/hosts

Erstellt einen DNS Eintrag (z.B. in AWS Route 53) wenn der Server via IP-Adresse im Internet verfügbar ist.

Ergänzt die Datei `/etc/hosts` wenn der Server nur im lokalen Netzwerk verfügbar ist.

### Routing in Kubernetes

Verwenden Sie den Anforderungs-Routing-Assistenten für den Steuerungsdienst , um eine Datenverkehrsregel zu generieren

![](https://kiali.io/images/tutorial/03-03-service-actions.png)

Verwenden Sie die Schaltfläche „Routing-Regel hinzufügen“, um eine Standardregel hinzuzufügen, bei der alle Anforderungen an die Kontroll- Workload weitergeleitet werden.

![](https://kiali.io/images/tutorial/03-03-request-routing.png)

Verwenden Sie die erweiterten Optionen, fügen Sie ein Gateway mit Host hinzu control.travel-control.istio-cluster.orgund erstellen Sie die Istio-Konfiguration.

![](https://kiali.io/images/tutorial/03-03-create-gateway.png)

Überprüfen Sie die generierte Istio-Konfiguration.

![](https://kiali.io/images/tutorial/03-03-istio-config.png)

[Original Dokumentation](https://kiali.io/docs/tutorials/travels/03-first-steps/)


### Testen

Testen Sie den Steuerungsdienst , in Ihrem Browser http://control.travel-control.istio-cluster.org (intern) oder bei einem externen DNS

![](https://kiali.io/images/tutorial/03-03-test-gateway.png)

In [ ]:
%%bash
PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo http://travel.mc-b.ch:$PORT/

Sehen Sie sich den Reisesteuerungs - Namespace in Kiali an

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

![](https://kiali.io/images/tutorial/03-03-travel-control-graph.png)